## DDQN in Hockey Enviroment - Evaluation

Evaluate a trained DDQN against various environments

Base DQN implementation adapted from HW7

In [6]:
from importlib import reload

import numpy as np
from tqdm.notebook import tqdm

import DDQN.DDQN as ddqn
from DDQN.DDQN import DDQNAgent
from DDQN.evaluation import compare_agents, display_stats
import hockey.hockey_env as h_env

reload(h_env)
reload(ddqn)

/home/kivanc/miniconda3/envs/rl/lib/python3.11/site-packages/gymnasium/envs/registration.py:642: UserWarning: WARN: Overriding environment Hockey-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/kivanc/miniconda3/envs/rl/lib/python3.11/site-packages/gymnasium/envs/registration.py:642: UserWarning: WARN: Overriding environment Hockey-One-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


<module 'DDQN.DDQN' from '/home/kivanc/ders/RL/project/RL-Hockey/DDQN/DDQN.py'>

In [7]:
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

## Environment & Agent Initialization

In [8]:
env = h_env.HockeyEnv(mode=h_env.Mode.NORMAL)
env.reset()

(array([-3.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  3.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.41269159,  0.46165037,  0.        ,
         0.        ,  0.        ,  0.        ]),
 {'winner': 0,
  'reward_closeness_to_puck': 0.0,
  'reward_touch_puck': 0.0,
  'reward_puck_direction': 0.0})

In [10]:
STATE_PATH = "./DDQN/models/hockey_normal_dev/"

agent_player = DDQNAgent(
    env.observation_space,
    env.discrete_action_space
)
agent_player.load_state(STATE_PATH)

agent_opp_weak = h_env.BasicOpponent(weak=True)
agent_opp_strong = h_env.BasicOpponent(weak=False)

## Evaluation

### Winning Rates Against Opponents & Match Statistics

In [11]:
weak_opp_stats = compare_agents(
    agent_player, agent_opp_weak, env, num_matches=1000, tqdm=tqdm
)
strong_opp_stats = compare_agents(
    agent_player, agent_opp_strong, env, num_matches=1000, tqdm=tqdm
)

print("Weak Opponent Stats:")
display_stats(weak_opp_stats)
print("Strong Opponent Stats:")
display_stats(strong_opp_stats)

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Weak Opponent Stats:
Player Observation Mean:
  x pos player one: -1.341544129920022
  y pos player one: -0.5941261902834267
  angle player one: -0.07263824568604207
  x vel player one: 0.8476032633522432
  y vel player one: -0.46109484363481845
  angular vel player one: -0.09071381830290962
  x player two: 2.581570860655187
  y player two: 0.067045480248056
  angle player two: -0.008268813406722936
  y vel player two: -0.0728967898016591
  y vel player two: -0.003303426745194766
  angular vel player two: 0.00330208470724013
  x pos puck: -0.05532382758773605
  y pos puck: 0.14756710077422897
  x vel puck: -1.1185261813095293
  y vel puck: 0.06893878195582148
  left player puck keep time: 0.8867102396514162
  right player puck keep time: 0.8948481353325644

Relative Std. Change in Agent Observations:
  x pos player one: -0.1401811667498876
  y pos player one: 0.5784090744911511
  angle player one: 0.7725071652330459
  x vel player one: -0.18573885630187922
  y vel player one: 0.6229716

### Rendered Demonstration Against Strong Opponent

In [ ]:
obs_buffer = []
reward_buffer = []
obs, _ = env.reset()
obs_opp = env.obs_agent_two()

done = False
trunc = False
step = 0
while not (done or trunc):
    step += 1
    env.render()

    a1_discr = agent_player.act(obs)
    a1 = env.discrete_to_continous_action(a1_discr)
    a2 = agent_opp_strong.act(obs_opp)

    obs, r, done, trunc, _ = env.step(np.hstack([a1, a2]))
    obs_buffer.append(obs)
    reward_buffer.append(r)

    obs_opp = env.obs_agent_two()

    if done or trunc:
        print(f"Episode done in {step} steps")
        break

Episode done in 171 steps


In [ ]:
env.close()